In [30]:
import csv
import pandas_datareader as web
import yfinance as yf
import sys 
#import quandl
import numpy as np
from sklearn .linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import datetime
import pandas_datareader as web
import yahoofinance
import pandas 

In [31]:
data = pandas.read_csv('dataset.csv',sep =";", error_bad_lines=False)
pandas.DataFrame(data)

,symbol,shortName,address1,address2,city,state,zip,country,phone,fax,website,industry,sector,fullTimeEmployees,maxAge
0,AIR.PA,NaN,Mendelweg 30,NaN,Leiden,NaN,2333 CS,Netherlands,31 71 524 5600,NaN,http://www.airbus.com,Aerospace & Defense,Industrials,131349.0,86400
1,MT.AS,NaN,"24-26, Boulevard dâAvranches",NaN,Luxembourg City,NaN,1160,Luxembourg,352 4792 1,352 4792 2235,http://corporate.arcelormittal.com,Steel,Basic Materials,168000.0,86400
2,CS.PA,NaN,"25, avenue Matignon",NaN,Paris,NaN,75008,France,33 1 40 75 57 00,NaN,http://www.axa.com,Insurance—Diversified,Financial Services,110278.0,86400
3,BNP.PA,NaN,16 boulevard des Italiens,NaN,Paris,NaN,75009,France,33 1 40 14 45 46,NaN,http://group.bnpparibas,Banks—Regional,Financial Services,193319.0,86400
4,ALO.PA,NaN,"48, rue Albert Dhalenne",NaN,St Ouen,NaN,93400,France,33 1 57 06 90 00,33 1 57 06 96 66,http://www.alstom.com,Specialty Industrial Machinery,Industrials,38900.0,86400
5,CA.PA,NaN,"26, Quai Michelet",NaN,Levallois-Perret,NaN,92695,France,33 1 55 63 39 00,33 1 55 63 39 01,http://www.carrefour.com,Grocery Stores,Consumer Defensive,NaN,86400
6,HO.PA,NaN,Tour Carpe Diem,Place des Corolles Esplanade Nord,Courbevoie,NaN,92400,France,33 1 57 77 80 00,NaN,http://www.thalesgroup.com/en,Aerospace & Defense,Industrials,80702.0,86400
7,DSY.PA,NaN,"10, Rue Marcel Dassault",NaN,VÃ©lizy-Villacoublay,NaN,78140,France,33 1 61 62 61 62,33 1 70 73 43 63,http://www.3ds.com,Software—Application,Technology,19789.0,86400
8,OR.PA,NaN,"41, rue Martre",NaN,Clichy,NaN,92117,France,33 1 47 56 70 00,NaN,http://www.loreal.com,Household & Personal Products,Consumer Defensive,85392.0,86400
9,VIV.PA,NaN,42 avenue de Friedland,Cedex 08,Paris,NaN,75380,France,33 1 71 71 10 00,NaN,http://www.vivendi.com,Broadcasting,Communication Services,42800.0,86400


In [32]:
start = datetime.datetime(2021,1,1) #année/ mois / jour
end = datetime.datetime(2021,3,30)
CAC40= ['AI.PA','AIR.PA','ALO.PA','MT.AS','CS.PA','BNP.PA','EN.PA', 'CAP.PA' ,'CA.PA','ACA.PA','BN.PA','DSY.PA','ENGI.PA','EL.PA','RMS.PA','KER.PA','OR.PA','LR.PA','MC.PA','ML.PA','ORA.PA','RI.PA','PUB.PA','RNO.PA','SAF.PA','SGO.PA','SAN.PA','SU.PA','GLE.PA','STLA.PA','STM.PA','TEP.PA','HO.PA','FP.PA','URW.PA','VIE.PA','DG.PA','VIV.PA','WLN.PA']

In [33]:
#for i, c in enumerate(CAC40):
df = web.DataReader(CAC40[5],'yahoo',start, end)


#help(yahoofinance.BalanceSheet)
print(type(df))

df = pandas.DataFrame(df)



<class 'pandas.core.frame.DataFrame'>


In [34]:

forecast_out = 30 #nb jours

df=df[['Adj Close']]

df['Prediction']= df.shift(-forecast_out)
df

,Adj Close,Prediction
Date,,
2021-01-04,43.009998,47.549999
2021-01-05,42.919998,47.610001
2021-01-06,45.290001,47.650002
2021-01-07,46.310001,47.075001
2021-01-08,45.285000,48.325001
...,...,...
2021-03-24,51.119999,NaN
2021-03-25,50.830002,NaN
2021-03-26,51.669998,NaN


In [35]:
"""forecast_out = 30 #nb jours
df['Prediction']= df[['Adj. Close']].shift(-forecast_out)
print(df.tail())"""

"forecast_out = 30 #nb jours\ndf['Prediction']= df[['Adj. Close']].shift(-forecast_out)\nprint(df.tail())"

In [36]:
X = np.array(df.drop(['Prediction'],1))
X = X[:-forecast_out]
y = np.array(df['Prediction'])
y = y[:-forecast_out]
print(X,'\n',y)

[[43.00999832]
 [42.91999817]
 [45.29000092]
 [46.31000137]
 [45.28499985]
 [44.97999954]
 [45.34000015]
 [45.02500153]
 [45.13000107]
 [44.43500137]
 [44.80500031]
 [44.11000061]
 [44.41999817]
 [44.34000015]
 [43.08499908]
 [41.22999954]
 [41.99000168]
 [40.52500153]
 [41.20500183]
 [39.77999878]
 [40.18000031]
 [41.60499954]
 [41.90000153]
 [42.48500061]
 [43.59500122]
 [44.11999893]
 [43.91999817]
 [45.13499832]
 [45.69499969]
 [46.47999954]
 [47.54999924]
 [47.61000061]] 
 [47.54999924 47.61000061 47.65000153 47.07500076 48.32500076 49.29999924
 49.45000076 50.16999817 50.31000137 49.27500153 49.13499832 50.45000076
 51.72000122 51.65000153 51.59000015 53.25       52.59000015 52.75999832
 52.33000183 53.08000183 51.83000183 51.18999863 51.68000031 52.88999939
 51.15000153 50.70999908 50.36000061 51.11999893 50.83000183 51.66999817
 50.66999817 52.61999893]


In [37]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = .8)
#help(SVR)

In [28]:
svr_rbf = SVR(kernel ='rbf', C =1e3, gamma = .20)
C = np.exp(1*np.arange(-5,15))
for c in C:
    svr_rbf = SVR(kernel ='rbf', C =c, gamma = 20)
    svr_rbf.fit(x_train, y_train)
    svm_confidence = svr_rbf.score(x_test, y_test)
    print(f"score pour c = {c} svm: {svm_confidence}")
sve_rbf = SVR(kernel ='rbf', C =7.31, gamma = .20)

score pour c = 0.006737946999085467 svm: -0.14487387615071423
score pour c = 0.01831563888873418 svm: -0.1449986218351491
score pour c = 0.049787068367863944 svm: -0.14545530356044356
score pour c = 0.1353352832366127 svm: -0.14321393100454638
score pour c = 0.36787944117144233 svm: -0.09011331338146822
score pour c = 1.0 svm: 0.01709316548312889
score pour c = 2.718281828459045 svm: 0.06431121445725296
score pour c = 7.38905609893065 svm: 0.10160661358401735
score pour c = 20.085536923187668 svm: 0.10160661358401735
score pour c = 54.598150033144236 svm: 0.10160661358401735
score pour c = 148.4131591025766 svm: 0.10160661358401735
score pour c = 403.4287934927351 svm: 0.10160661358401735
score pour c = 1096.6331584284585 svm: 0.10160661358401735
score pour c = 2980.9579870417283 svm: 0.10160661358401735
score pour c = 8103.083927575384 svm: 0.10160661358401735
score pour c = 22026.465794806718 svm: 0.10160661358401735
score pour c = 59874.14171519782 svm: 0.10160661358401735
score pou

In [29]:
svm_confidence = svr_rbf.score(x_test, y_test)
print("score svm: ", svm_confidence)

score svm:  0.10160661358401735
